# HW3: Document retrieval of the AICUP2023 competition
The goal of this homework is to implement a document retrieval system for the AICUP2023 competition using the [`TF-IDF`](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) method.

## What is TF-IDF?
- TF: Term Frequency, the occurring time ($n$) of `a term` $t$ appears in `a document` $d$.
$$\textrm{TF}_{t,d} =  \frac{n_{t,d}}{\sum_k n_{k,d}}$$
, where $\sum_k n_{k,d}$ indicates summation of all terms in the document $d$.
- IDF: Inverse Document Frequency, how frequent `a term` $t$ appears in all documents.
    - Document Frequency $\textrm{DF}_t = \log\frac{|\{d: t\in d\}|}{|D|}$, where $|\{d: t\in d\}|$ is the number of documents that `a term` t appears, and $|D|$ indicates total number of documents.
    - $\Rightarrow \textrm{IDF}_t = \log\frac{|D|}{|\{d: t\in d\}|}$
- $\textrm{TFIDF}_{t,d} = \textrm{TF}_{t,d} \times \textrm{IDF}_t$

## What can TF-IDF do?
- TF-IDF can be used to transform a document or a sentence into a `vector`.
- There are other ways to transform a document or a sentence into a vector, such as [`word2vec`](https://radimrehurek.com/gensim/models/word2vec.html), [`BERT`](https://huggingface.co/docs/transformers/training), [`Sentence-BERT`](https://github.com/UKPLab/sentence-transformers), etc.
- **This homework only asks you to implement the TF-IDF method.** You can try other methods in your final project.

## Example of TF-IDF using `scikit-learn`

### Example 1: Use [`CountVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) + [`TfidfTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html)
```python
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

# Assume we have 4 documents in a list
corpus = [
    "this is the first document",
    "this document is the second document",
    "and this is the third one",
    "is this the first document",
]
# Assume we want some important words in the vocabulary
vocabulary = ["this", "document", "first", "is", "second", "the", "and", "one"]
pipe = Pipeline(
    [
        ("count", CountVectorizer(vocabulary=vocabulary)),  # Get the count of each word
        ("tfidf", TfidfTransformer(norm=None)),  # Get the tfidf of each word
    ]
).fit(corpus)

# Print the count of each word
print(pipe["count"].transform(corpus).toarray())
>>> [[1, 1, 1, 1, 0, 1, 0, 0], # count of the first document
     [1, 2, 0, 1, 1, 1, 0, 0], # count of the second document
     [1, 0, 0, 1, 0, 1, 1, 1], # count of the third document
     [1, 1, 1, 1, 0, 1, 0, 0]] # count of the fourth document

# Print the IDF values of each word
# Notice!! The shape of the IDF values is (n_features, ).
# n_features is the number of words in the vocabulary.
print(pipe["tfidf"].idf_)
>>> [1.         1.22314355 1.51082562 1.         1.91629073 1.
 1.91629073 1.91629073]

# Print the TF-IDF values
# Notice!! The shape of the TF-IDF values is (n_documents, n_features).
print(pipe.transform(corpus).toarray())
>>> [[1.         1.22314355 1.51082562 1.         0.         1.
  0.         0.        ]
 [1.         2.4462871  0.         1.         1.91629073 1.
  0.         0.        ]
 [1.         0.         0.         1.         0.         1.
  1.91629073 1.91629073]
 [1.         1.22314355 1.51082562 1.         0.         1.
  0.         0.        ]]
```

### Example 2: Use [`TfidfVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) alone
```python
from sklearn.feature_extraction.text import TfidfVectorizer

# Assume we have 4 documents in a list
corpus = [
    "this is the first document",
    "this document is the second document",
    "and this is the third one",
    "is this the first document",
]
# Assume we want some important words in the vocabulary
vocabulary = ["this", "document", "first", "is", "second", "the", "and", "one"]
vectorizer = TfidfVectorizer(
    vocabulary=vocabulary,
    norm=None, # without normalization
)
X = vectorizer.fit_transform(corpus)
print(X.toarray())
>>> [[1.         1.22314355 1.51082562 1.         0.         1.
  0.         0.        ]
 [1.         2.4462871  0.         1.         1.91629073 1.
  0.         0.        ]
 [1.         0.         0.         1.         0.         1.
  1.91629073 1.91629073]
 [1.         1.22314355 1.51082562 1.         0.         1.
  0.         0.        ]]
```
- Example 2 is the same as Example 1, but we use `TfidfVectorizer` instead of `CountVectorizer` + `TfidfTransformer`.

## Why do we need to transform a document or a sentence into a vector?
- Because we can use the `cosine similarity` to measure the similarity between each sentence and a Wikipedia article.
- In our AICUP2023 task, we can find the most similar Wikipedia article for each `claim`.

## HW3 Delivery policies
|Rules | Punishment | Note |
|-|-|-|
| Name the folder as `FDA_HW3_F12345678` and zip it | -5| |
| Include `requirements.txt` in the folder | -5 | |
| Use ChatGPT to predict the answers (not allowed in HW3) | -20 | You can use ChatGPT to help coding work. |
| Results cannot be reproduced by re-running | -10 | We will run your code! |
|Explicit rule-based corrections (e.g. predictions[0] = ["天衛三"]) | -20 |  |
|Copy and paste the code from your classmates | -20 |  |
|Modify the `do-not-modify cell` | -20 |  |

## HW3 Scoring rules
|Rules | Scores|
|-|-|
| Perform document retrieval with scikit-learn `TF-IDF` by finishing all # TODO (each for +16) | +80 |
| Document retrieval Precision >= 0.25 and Recall >= 0.6 (on the training set) | +10 |
| Document retrieval Precision >= 0.4 and Recall >= 0.7 (on the training set) | +10 |
| (Bonus) Document retrieval Precision >= 0.5 and Recall >= 0.7 (on the training set) | +10 |
| (Bonus) Document retrieval Precision >= 0.6 and Recall >= 0.7 (on the training set) | +10 |

## Before you start
1. Join the AICUP2023 competition. Link: https://tbrain.trendmicro.com.tw/Competitions/Details/28
2. Download the training data and the Wikipedia data.
3. (This code will need memory about 5.7 GB, tested on Google Colab.)

### Folder structure
- Please follow the following folder structure.
```bash
FDA_HW3_F12345678/
│
├── data/ 
│ ├── public_train.jsonl # Downloaded from the AICUP2023 page
│ └── wiki-pages # Downloaded from the AICUP2023 page and zipped
│
├── FDA2023_HW3.ipynb # This file
├── hw3_utils.py
└── requirements.txt
```

In [1]:
# Uncomment the following lines if you use Google Colab
# !pip install pandarallel
# !pip install TCSP

In [2]:
from pathlib import Path
from functools import partial
import re
import numpy as np
import pandas as pd
import jieba
import scipy

# jieba.set_dictionary("dict.txt.big")
# Download "dict.txt.big" from https://github.com/fxsjy/jieba

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from pandarallel import pandarallel
# Adjust the number of workers if you want
pandarallel.initialize(progress_bar=True, verbose=0, nb_workers=3)

from tqdm import tqdm
tqdm.pandas() # for progress_apply

from hw3_utils import (
    load_json,
    jsonl_dir_to_df,
    calculate_precision,
    calculate_recall,
)

In [3]:
# Get the stopwords
# https://github.com/bryanchw/Traditional-Chinese-Stopwords-and-Punctuations-Library
from TCSP import read_stopwords_list

stopwords = read_stopwords_list()

In [4]:
def tokenize(text: str, stopwords: list) -> str:
    import jieba
    """This function performs Chinese word segmentation and removes stopwords.

    Args:
        text (str): claim or wikipedia article
        stopwords (list): common words that contribute little to the meaning of a sentence

    Returns:
        str: word segments separated by space (e.g. "我 喜歡 吃 蘋果")
    """

    tokens = jieba.cut(text)

    return " ".join([w for w in tokens if w not in stopwords])


In [23]:
def get_pred_docs_sklearn(
    claim: str,
    tokenizing_method: callable,
    vectorizer: TfidfVectorizer,
    tf_idf_matrix: scipy.sparse.csr_matrix,
    wiki_pages: pd.DataFrame,
    topk: int,
) -> set:
    import numpy as np

    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    
    tokens = tokenizing_method(claim)
    claim_vector = vectorizer.transform([tokens])
    similarity_scores = cosine_similarity(tf_idf_matrix, claim_vector)

    # `similarity_scores` shape: (num_wiki_pages x 1)
    similarity_scores = similarity_scores[:, 0]  # flatten the array

    # Sort the similarity scores in descending order
    sorted_indices = np.argsort(similarity_scores)[::-1]
    topk_sorted_indices = sorted_indices[:topk]

    # Get the wiki page names based on the topk sorted indices 
    results = wiki_pages.iloc[topk_sorted_indices]["id"]

    exact_matchs = []
    # You can find the following code in our AICUP2023 baseline.
    # Basically, we check if a result is exactly mentioned in the claim.
    for result in results:
        if (
            (result in claim)
            or (result in claim.replace(" ", "")) # E.g., MS DOS -> MSDOS
            or (result.replace("·", "") in claim) # E.g., 湯姆·克魯斯 -> 湯姆克魯斯
            or (result.replace("-", "") in claim) # E.g., X-SAMPA -> XSAMPA
        ):
            exact_matchs.append(result)
        elif "·" in result:
            splitted = result.split("·") # E.g., 阿爾伯特·愛因斯坦 -> 愛因斯坦
            for split in splitted:
                if split in claim:
                    exact_matchs.append(result)
                    break

    return set(exact_matchs)

In [6]:
# Helper function (you don't need to modify this)

def get_title_from_evidence(evidence):
    titles = []
    for evidence_set in evidence:
        if len(evidence_set) == 4 and evidence_set[2] is None:
            return [None]
        for evidence_sent in evidence_set:
            titles.append(evidence_sent[2])
    return list(set(titles))


def save_results_to_markdown(results: dict, output_file="grid_search_results.md"):
    file_exists = Path(output_file).exists()

    with open(output_file, "a") as f:
        if not file_exists:
            f.write("# Grid Search Results\n\n")
            f.write("| Experiment  | F1 Score | Precision | Recall |\n")
            f.write("| ----------- | -------- | --------- | ------ | \n")

        exp_name = results["exp_name"]
        f1 = results["f1_score"]
        prec = results["precision"]
        recall = results["recall"]
        f.write(f"| {exp_name} | {f1:.4f} | {prec:.4f} | {recall:.4f} |\n")
    print(f"Results saved to {output_file}")

In [7]:
# Hyperparameters

wiki_path = "data/wiki-pages"
min_wiki_length = 10
num_of_samples = 500
topk = 15
min_df = 2
max_df = 0.8
use_idf = True
sublinear_tf = True

# Set up the experiment name for logging
exp_name = (
    f"len{min_wiki_length}_top{topk}_min_df={min_df}_"
    + f"max_df={max_df}_{num_of_samples}s"
)
if sublinear_tf:
    exp_name = "sublinearTF_" + exp_name
if not use_idf:
    exp_name = "no_idf_" + exp_name

In [8]:
# First time running this cell will 34 minutes using Google Colab.
wiki_cache = "wiki"
target_column = "text"

wiki_cache_path = Path(f"data/{wiki_cache}.pkl")
if wiki_cache_path.exists():
    wiki_pages = pd.read_pickle(wiki_cache_path)
else:
    # You need to download `wiki-pages.zip` from the AICUP website
    wiki_pages = jsonl_dir_to_df(wiki_path)
    # wiki_pages are combined into one dataframe, so we need to reset the index
    wiki_pages = wiki_pages.reset_index(drop=True)

    # tokenize the text and keep the result in a new column `processed_text`
    wiki_pages["processed_text"] = wiki_pages[target_column].parallel_apply(
        partial(tokenize, stopwords=stopwords)
    )
    # save the result to a pickle file
    wiki_pages.to_pickle(wiki_cache_path, protocol=4)

In [9]:
# Build the TfidfVectorizer

vectorizer = TfidfVectorizer(
    min_df=min_df,
    max_df=max_df,
    use_idf=use_idf,
    sublinear_tf=sublinear_tf,
    dtype=np.float64,
)

In [10]:
wiki_pages = wiki_pages[
    wiki_pages['processed_text'].str.len() > min_wiki_length
]
corpus = wiki_pages["processed_text"].tolist()

In [11]:
X = vectorizer.fit_transform(corpus)

# fit_transform will do the following two steps:
# 1. fit: learn the vocabulary and idf from the corpus
# 2. transform: transform the corpus into a vector space
# Note the result is a sparse matrix, which contains lots of zeros for each row.

In [12]:
train = load_json("data/public_train.jsonl")[:num_of_samples]
train_df = pd.DataFrame(train)

In [24]:
# Perform the prediction for document retrieval
# If you use Google Colab, do not use parallel_apply due to the memory limit.

train_df["predictions"] = train_df["claim"].apply(
# train_df["predictions"] = train_df["claim"].progress_apply(
    partial(
        get_pred_docs_sklearn,
        tokenizing_method=partial(tokenize, stopwords=stopwords),
        vectorizer=vectorizer,
        tf_idf_matrix=X,
        wiki_pages=wiki_pages,
        topk=topk,
    )
)

995170             緹坦妮雅
186143      仲夏夜之夢_(消歧義)
197396           尼克·博特姆
995169              奧伯隆
9661                天衛三
122018               磁層
1136853      仲夏夜_(羅文專輯)
857790         托馬斯·卡拉布羅
380707         仙后_(消歧義)
367848           小行星685
81583               磁層頂
213220     結婚進行曲_(孟德爾頌)
757629     仲夏夜之夢_(門德爾松)
350491           小行星593
860607         天王星軌道探測器
Name: id, dtype: object
995092       桑氏遠洋鳥
203406          翼展
200382       漂泊信天翁
192384     南方皇家信天翁
200171       特島信天翁
8207          信天翁科
625819       黑背信天翁
816800       灰背信天翁
315437       黑眉信天翁
228700       黑腳信天翁
959856    智慧_(信天翁)
8205         大信天翁屬
229551          黃雀
228766          大鵟
8202          白嘴潛鳥
Name: id, dtype: object
40469       F.I.R.飛兒樂團
131632           梵谷的左耳
396998      建寧街道_(曲靖市)
568010     紀念日_(炎亞綸專輯)
724663     Better_Life
280117            Real
201270        浮雲_(吉他手)
131596              阿沁
385750             宋雨哲
287769      宇宙人_(臺灣樂團)
92218              俞飛鴻
875021             任晙赫
188359          阿姆迪·法耶
163168  

KeyboardInterrupt: 

In [14]:
precision = calculate_precision(train, train_df["predictions"])
recall = calculate_recall(train, train_df["predictions"])
results = {
    "exp_name": exp_name,
    "f1_score": 2.0 * precision * recall / (precision + recall),
    "precision": precision,
    "recall": recall,
}

# This helps you to adjust the hyperparameters
save_results_to_markdown(results, output_file="grid_search_results.md")

Precision: 0.4978260044049519
Recall: 0.6675438596491229
Results saved to grid_search_results.md


In [15]:
# Do not modify this cell.
# This cell is for your scores on the training set.

train = load_json("data/public_train.jsonl")
train_df = pd.DataFrame(train)

pandarallel.initialize(progress_bar=True, verbose=0, nb_workers=4)
# Perform the prediction for document retrieval
train_df["predictions"] = train_df["claim"].parallel_apply(
    partial(
        get_pred_docs_sklearn,
        tokenizing_method=partial(tokenize, stopwords=stopwords),
        vectorizer=vectorizer,
        tf_idf_matrix=X,
        wiki_pages=wiki_pages,
        topk=topk,
    )
)
precision = calculate_precision(train, train_df["predictions"])
recall = calculate_recall(train, train_df["predictions"])

Precision: 0.4779389682176204
Recall: 0.6322995382031905
